# Academic Papers RAG System Tutorial

This notebook demonstrates how to build a complete RAG (Retrieval-Augmented Generation) system for academic research papers using LlamaIndex. We'll build it step by step with independent functions that you can run and understand individually.

## What is RAG?

RAG combines the power of:
- **Retrieval**: Finding relevant documents from a knowledge base
- **Augmented Generation**: Using retrieved context to generate informed responses

## System Components

Our RAG system will include:
1. **PDF Processing**: Extract text from academic papers
2. **Document Chunking**: Split documents into searchable segments
3. **Vector Embeddings**: Convert text to numerical representations
4. **Vector Storage**: Store embeddings in LanceDB
5. **Semantic Search**: Find relevant content for queries
6. **Query Engine**: Generate responses using retrieved context


## 🏗️ Storage Architecture: Why StorageContext Matters

This notebook uses LlamaIndex's **StorageContext** approach, which provides significant advantages over simpler vector-only storage methods. Understanding this architecture is crucial for building production-ready RAG systems.

### 📊 StorageContext vs. Simple Vector Storage

| Component | StorageContext (This Notebook) | Simple Vector Store | Benefits |
|-----------|-------------------------------|-------------------|----------|
| **Vector Store** | ✅ LanceDB embeddings | ✅ LanceDB embeddings | Fast similarity search |
| **Document Store** | ✅ Original documents preserved | ❌ Lost after processing | Full document access |
| **Index Store** | ✅ Index metadata & structure | ❌ Must rebuild index | Exact reconstruction |
| **Graph Store** | ✅ Document relationships | ❌ No relationship data | Rich context understanding |

### 🔄 Persistence & Recovery Capabilities

**With StorageContext (Our Approach):**
```python
# Save complete system state
index.storage_context.persist(persist_dir="storage/papers_index")

# Perfect restoration - identical behavior
storage_context = StorageContext.from_defaults(persist_dir="storage/papers_index")
index = load_index_from_storage(storage_context)
# 🎯 Exact same results every time!
```

**Simple Vector Store Only:**
```python
# Only vectors saved
vector_store = LanceDBVectorStore(uri="./vectors")

# Must recreate everything from scratch
index = VectorStoreIndex.from_vector_store(vector_store)
# ⚠️ May have different behavior, lost metadata
```

### 💡 Key Advantages of StorageContext

1. **🔄 Perfect Reproducibility**: Identical results across sessions - critical for research and development
2. **📦 Complete State Management**: All components preserved, not just vectors
3. **⚡ Fast Startup**: No reprocessing needed - load existing index instantly
4. **🔍 Rich Metadata**: Document relationships, source tracking, and complex queries
5. **🛠️ Development Friendly**: Iterate without rebuilding entire system
6. **🎯 Enterprise Ready**: Robust persistence for production deployments

### 📈 Storage Footprint Example

For 1000 academic papers (~500MB original PDFs):

**StorageContext Storage:**
```
storage/papers_index/
├── docstore.json          # 50MB - Original documents
├── index_store.json       # 5MB  - Index metadata  
├── graph_store.json       # 2MB  - Relationships
└── LanceDB vector files   # 200MB - Embeddings
Total: ~260MB
```

**Benefits**: Complete system restoration, full metadata, relationships preserved

**Simple Vector Storage:**
```
lancedb_data/
└── vectors.lance          # 200MB - Embeddings only
Total: ~200MB
```

**Limitations**: Must rebuild index, lost metadata, no relationships

### 🎯 When to Use StorageContext

✅ **Research & Development** - Need reproducible experiments  
✅ **Complex Documents** - Rich metadata and relationships matter  
✅ **Production Systems** - Robust persistence and recovery required  
✅ **Academic Work** - Full traceability and citation tracking  
✅ **Multi-user Systems** - Consistent experience across users  

### 🚀 Performance Impact

- **Initial Build**: ~20% slower (stores additional metadata)
- **Subsequent Loads**: 10x faster (no reprocessing needed)
- **Query Performance**: Identical to simple vector approach
- **Storage Space**: ~30% more storage for complete persistence

This tutorial demonstrates the StorageContext approach because it provides the most robust and feature-complete RAG implementation suitable for real-world applications.


## 1. Environment Setup and Configuration

First, let's set up our environment and load necessary configurations. We'll use OpenRouter for LLM access and local embeddings (no API keys needed for embeddings).


In [ ]:
# We need mount our Google Drive so that our Colab notebooks can access content.
# Here we will need to pip install all the requirements.
# Later in the code we will require access to all documents we want stored for RAG.

import os
from google.colab import drive

# 1️⃣ Mount Google Drive
drive.mount('/content/drive')

# 2️⃣ Define your project folder inside Drive
project_path = "/content/drive/MyDrive/Colab Notebooks"

# 3️⃣ Construct the requirements path dynamically
requirements_path = os.path.join(project_path, "requirements.txt")

# 4️⃣ Validate the file exists
if not os.path.exists(requirements_path):
    raise FileNotFoundError(
        f"❌ Could not find requirements.txt at expected path:\n{requirements_path}\n"
        "Please verify your Google Drive folder structure or update project_path."
    )
else:
    print(f"✅ Found requirements.txt at: {requirements_path}")

# 5️⃣ Install dependencies
!pip install -r "$requirements_path"

# !pip install -r "../requirements.txt"

Mounted at /content/drive
✅ Found requirements.txt at: /content/drive/MyDrive/Colab Notebooks/requirements.txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 19.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 14.7 MB/s eta 0:00:00
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you wa

In [ ]:
import os
import time
from pathlib import Path
from typing import Dict, List, Optional, Tuple

from dotenv import load_dotenv

def setup_environment():
    """
    Setup environment variables and basic configuration.

    Returns:
        bool: Success status
    """
    # Load environment variables from .env file
    load_dotenv()

    # Disable tokenizer warning
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    # Check for required API key
    # api_key = os.getenv("OPENROUTER_API_KEY")

    from google.colab import userdata
    api_key = userdata.get('OPENROUTER_API_KEY')
    # print ("The OpenRouter API Key is ", api_key)

    if not api_key:
        print("⚠️  OPENROUTER_API_KEY not found in environment variables")
        print("Please add your OpenRouter API key to a .env file")
        return False

    print("✓ Environment variables loaded successfully")
    return True

# Run the setup
success = setup_environment()
if success:
    print("Environment setup complete!")
else:
    print("Environment setup failed!")

✓ Environment variables loaded successfully
Environment setup complete!


## 2. Configuration Management

Let's define our system configuration directly in the notebook. This includes model settings, chunk sizes, and other parameters.


In [ ]:
# Configuration parameters for the RAG system

import os
from google.colab import drive
# 1️⃣ Mount Google Drive
drive.mount('/content/drive')
# 2️⃣ Define your project folder inside Drive
project_path = "/content/drive/MyDrive/Colab Notebooks"
print(f"Project path: {project_path}")
# 3️⃣ Construct the papers path dynamically
papers_folder = os.path.join(project_path, "papers-agents")
print(f"Papers folder: {papers_folder}")
# 4️⃣ Validate the file exists
if not os.path.exists(papers_folder):
    raise FileNotFoundError(
        f"❌ Could not find papers_folder at expected path:\n{papers_folder}\n"
        "Please verify your Google Drive folder structure or update project_path."
    )
else:
    print(f"✅ Found papers folder at: {papers_folder}")


CONFIG = {
    "llm": {
        "model": "gpt-4o",                    # OpenRouter model to use
        "temperature": 0.1                   # Temperature for response generation
    },
    "embeddings": {
        "model": "local:BAAI/bge-small-en-v1.5",  # Local embedding model (no API key needed)
        "chunk_size": 1024,                  # Size of text chunks for processing
        "chunk_overlap": 100                 # Overlap between consecutive chunks
    },
    "vector_store": {
        "type": "lancedb",                   # Vector database type
        "table_name": "academic_papers",     # Table name for storing embeddings
        "path": "storage/papers_vectordb"    # Path to vector database
    },
    "index": {
        "storage_path": "storage/papers_index",  # Path to store complete index
        "similarity_top_k": 5                    # Number of similar chunks to retrieve
    },
    "papers": {
 #       "folder": "/papers-agents"      # Path to academic papers folder
        "folder": os.path.join(project_path, "papers-agents")      # Path to academic papers folder
    }
}

def get_config(key_path: str, default_value=None):
    """
    Get configuration value using dot notation.

    Args:
        key_path (str): Dot-separated path to the config value (e.g., 'llm.model')
        default_value: Default value if key not found

    Returns:
        Configuration value or default
    """
    keys = key_path.split('.')
    value = CONFIG

    for key in keys:
        if isinstance(value, dict) and key in value:
            value = value[key]
        else:
            return default_value

    return value

# Test configuration access
llm_model = get_config("llm.model")
embedding_model = get_config("embeddings.model")
chunk_size = get_config("embeddings.chunk_size")

print(f"LLM model: {llm_model}")
print(f"Embedding model: {embedding_model}")
print(f"Chunk size: {chunk_size}")
print("✓ Configuration setup complete")

papers_folder = get_config("papers.folder")
print(f"Loading papers from: {papers_folder}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Project path: /content/drive/MyDrive/Colab Notebooks
Papers folder: /content/drive/MyDrive/Colab Notebooks/papers-agents
✅ Found papers folder at: /content/drive/MyDrive/Colab Notebooks/papers-agents
LLM model: gpt-4o
Embedding model: local:BAAI/bge-small-en-v1.5
Chunk size: 1024
✓ Configuration setup complete
Loading papers from: /content/drive/MyDrive/Colab Notebooks/papers-agents


## 3. LlamaIndex Settings Configuration

LlamaIndex uses global settings for embeddings, LLMs, and document processing. We'll use OpenRouter for the LLM and a local embedding model (no API key required).


In [ ]:
from llama_index.core import Settings
from llama_index.llms.openrouter import OpenRouter
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.node_parser import SentenceSplitter

def configure_llamaindex_settings():
    """
    Configure LlamaIndex global settings for embeddings, LLM, and text processing.
    """
    # Set up LLM with OpenRouter
    model = get_config("llm.model")
    temperature = get_config("llm.temperature", 0.1)

    from google.colab import userdata
    api_key = userdata.get('OPENROUTER_API_KEY')
    if not api_key:
        print("⚠️  OPENROUTER_API_KEY not found in environment variables")
        print("Please add your OpenRouter API key to a .env file")
        return False

    Settings.llm = OpenRouter(
        api_key=api_key,
        model=model,
        temperature=temperature
    )
    print(f"✓ LLM configured: {model} (temperature: {temperature})")

    # Set up local embedding model (downloads locally first time, then cached)
    embedding_model = get_config("embeddings.model")
    Settings.embed_model = resolve_embed_model(embedding_model)
    print(f"✓ Embedding model configured: {embedding_model}")

    # Set up node parser for chunking
    chunk_size = get_config("embeddings.chunk_size")
    chunk_overlap = get_config("embeddings.chunk_overlap")

    Settings.node_parser = SentenceSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    print(f"✓ Text chunking configured: {chunk_size} chars with {chunk_overlap} overlap")

# Configure the settings using our hardcoded config
configure_llamaindex_settings()
print("✓ LlamaIndex settings configured successfully")


✓ LLM configured: gpt-4o (temperature: 0.1)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Embedding model configured: local:BAAI/bge-small-en-v1.5
✓ Text chunking configured: 1024 chars with 100 overlap
✓ LlamaIndex settings configured successfully


## 4. Vector Store Setup

We'll use LanceDB as our vector database to store document embeddings. LanceDB is a fast, serverless vector database that's perfect for RAG applications.


In [ ]:
from llama_index.vector_stores.lancedb import LanceDBVectorStore

def create_vector_store():
    """
    Create and configure LanceDB vector store using config settings.

    Returns:
        LanceDBVectorStore: Configured vector store
    """
    try:
        import lancedb

        # Get configuration values
        vector_db_path = get_config("vector_store.path")
        table_name = get_config("vector_store.table_name")

        # Create storage directory
        Path(vector_db_path).parent.mkdir(parents=True, exist_ok=True)

        # Connect to LanceDB
        db = lancedb.connect(str(vector_db_path))
        print(f"✓ Connected to LanceDB at: {vector_db_path}")

        # Create vector store
        vector_store = LanceDBVectorStore(
            uri=str(vector_db_path),
            table_name=table_name
        )
        print(f"✓ LanceDB vector store created (table: {table_name})")

        return vector_store

    except Exception as e:
        print(f"Error creating vector store: {e}")
        return None

# Create the vector store using config
vector_store = create_vector_store()
if vector_store:
    print("✓ Vector store setup complete")
else:
    print("❌ Vector store setup failed")


✓ Connected to LanceDB at: storage/papers_vectordb
✓ LanceDB vector store created (table: academic_papers)
✓ Vector store setup complete


## 5. PDF Processing and Document Loading

Now we'll create functions to load and process PDF files. We'll use LlamaIndex's native `SimpleDirectoryReader` which can handle PDFs directly without needing a custom processor.


In [ ]:
from llama_index.core import SimpleDirectoryReader

def load_papers_from_folder() -> List:
    """
    Load and process all PDF papers from the configured folder using LlamaIndex's native loader.

    Returns:
        List[Document]: Processed documents ready for indexing
    """
    papers_folder = get_config("papers.folder")
    print(f"Loading papers from: {papers_folder}")

    papers_path = Path(papers_folder)
    if not papers_path.exists():
        print(f"Papers folder does not exist: {papers_path}")
        return []

    # Use LlamaIndex's SimpleDirectoryReader to load PDFs
    # This natively handles PDF parsing, text extraction, and metadata
    documents = SimpleDirectoryReader(papers_folder).load_data()

    print(f"✓ Loaded {len(documents)} documents")
    return documents

# Load the papers using config
documents = load_papers_from_folder()
if documents:
    print(f"Successfully loaded {len(documents)} documents")
    print(f"First document preview: {documents[0].text[:200]}...")
    print(f"First document metadata: {documents[0].metadata}")
else:
    print("No documents loaded")


Loading papers from: /content/drive/MyDrive/Colab Notebooks/papers-agents
✓ Loaded 229 documents
Successfully loaded 229 documents
First document preview: AI Agents vs. Agentic AI: A Conceptual
Taxonomy, Applications and Challenges
Ranjan Sapkota∗‡, Konstantinos I. Roumeliotis †, Manoj Karkee ∗‡
∗Cornell University, Department of Environmental and Biolo...
First document metadata: {'page_label': '1', 'file_name': 'AI_Agents_vs_Agentic_AI.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/papers-agents/AI_Agents_vs_Agentic_AI.pdf', 'file_type': 'application/pdf', 'file_size': 3196781, 'creation_date': '2025-11-02', 'last_modified_date': '2025-11-02'}


## 6. Creating the Vector Index

The vector index is the core of our RAG system. It chunks documents, generates embeddings, and stores them in the vector database for efficient similarity search.


In [ ]:
from llama_index.core import StorageContext, VectorStoreIndex, load_index_from_storage

def create_vector_index(documents: List,
                       vector_store,
                       force_rebuild: bool = False):
    """
    Create or load a vector index from documents using config settings.

    Args:
        documents (List): Documents to index
        vector_store: LanceDB vector store
        force_rebuild (bool): Force rebuild even if index exists

    Returns:
        VectorStoreIndex: The created or loaded index
    """
    index_storage_path = get_config("index.storage_path")
    index_path = Path(index_storage_path)
    index_path.mkdir(parents=True, exist_ok=True)

    # Check if index already exists
    index_store_file = index_path / "index_store.json"

    if not force_rebuild and index_store_file.exists():
        print("📁 Loading existing index...")
        try:
            # Recreate storage context with vector store
            storage_context = StorageContext.from_defaults(
                persist_dir=str(index_path),
                vector_store=vector_store
            )

            # Load existing index
            index = load_index_from_storage(storage_context)
            print("✓ Successfully loaded existing index")
            return index

        except Exception as e:
            print(f"⚠️  Error loading existing index: {e}")
            print("Creating new index...")

    if not documents:
        print("❌ No documents to index")
        return None

    print("🔨 Creating new vector index...")
    start_time = time.time()

    # Create storage context with vector store
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # Create index with progress bar
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        show_progress=True
    )

    end_time = time.time()
    print(f"✓ Index created in {end_time - start_time:.2f} seconds")

    # Save index to storage
    print("💾 Saving index to storage...")
    index.storage_context.persist(persist_dir=str(index_path))
    print("✓ Index saved successfully")

    return index

# Create the vector index using config
index = create_vector_index(
    documents=documents,
    vector_store=vector_store,
    force_rebuild=False  # Set to True to force rebuild
)

if index:
    print("✓ Vector index ready for querying")
else:
    print("❌ Failed to create vector index")


🔨 Creating new vector index...


Parsing nodes:   0%|          | 0/229 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/401 [00:00<?, ?it/s]

✓ Index created in 7.57 seconds
💾 Saving index to storage...
✓ Index saved successfully
✓ Vector index ready for querying


## 7. Setting Up the Query Engine

The query engine combines a retriever (to find relevant documents) with an LLM (to generate responses). This is where the "Augmented Generation" part of RAG happens.


In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever

def setup_query_engine(index):
    """
    Setup the query engine for semantic search and response generation using config settings.

    Args:
        index: The vector index to query

    Returns:
        RetrieverQueryEngine: Configured query engine
    """
    if not index:
        print("❌ Index not available. Please create index first.")
        return None

    try:
        # Get similarity top k from config
        similarity_top_k = get_config("index.similarity_top_k")

        # Create retriever - this finds the most similar document chunks
        retriever = VectorIndexRetriever(
            index=index,
            similarity_top_k=similarity_top_k,
        )
        print(f"✓ Retriever configured to find top {similarity_top_k} similar chunks")

        # Create query engine - this combines retrieval with LLM generation
        query_engine = RetrieverQueryEngine(retriever=retriever)
        print("✓ Query engine setup successfully")

        return query_engine

    except Exception as e:
        print(f"❌ Error setting up query engine: {e}")
        return None

# Setup the query engine using config
query_engine = setup_query_engine(index)

if query_engine:
    print("🚀 RAG system is ready for queries!")
else:
    print("❌ Failed to setup query engine")


✓ Retriever configured to find top 5 similar chunks
✓ Query engine setup successfully
🚀 RAG system is ready for queries!


In [ ]:
def extract_paper_title_from_text(text: str, max_length: int = 200) -> str:
    """
    Extract the paper title from the document text.

    Args:
        text (str): Document text content
        max_length (int): Maximum length for title extraction

    Returns:
        str: Extracted title or fallback
    """
    if not text:
        return "Unknown Title"

    # Split into lines and clean them
    lines = [line.strip() for line in text.split('\n') if line.strip()]

    if not lines:
        return "Unknown Title"

    # Look for title patterns - usually the first substantial line
    # Skip very short lines, page numbers, headers
    for line in lines[:10]:  # Check first 10 lines
        # Skip lines that look like headers, page numbers, or metadata
        if (len(line) > 15 and
            not line.isdigit() and
            not line.startswith(('Page', 'arXiv:', 'doi:', 'http', 'www')) and
            not all(c.isupper() or c.isspace() for c in line) and  # Skip all-caps headers
            '.' in line or len(line) > 30):  # Likely a title if it has punctuation or is long

            # Clean up the title
            title = line.strip()

            # Remove common prefixes/suffixes
            prefixes_to_remove = ['Title:', 'Abstract:', 'Paper:', 'Research:']
            for prefix in prefixes_to_remove:
                if title.startswith(prefix):
                    title = title[len(prefix):].strip()

            # Truncate if too long
            if len(title) > max_length:
                title = title[:max_length].strip() + "..."

            return title

    # Fallback: use first non-empty line, truncated
    first_line = lines[0] if lines else "Unknown Title"
    if len(first_line) > max_length:
        first_line = first_line[:max_length].strip() + "..."

    return first_line

def extract_paper_authors_from_text(text: str) -> str:
    """
    Extract authors from the document text.

    Args:
        text (str): Document text content

    Returns:
        str: Extracted authors or "Unknown Authors"
    """
    if not text:
        return "Unknown Authors"

    lines = [line.strip() for line in text.split('\n') if line.strip()]

    # Look for author patterns in first 20 lines
    for i, line in enumerate(lines[:20]):
        # Skip the title line (usually first substantial line)
        if i == 0:
            continue

        # Look for author patterns
        if (len(line) > 5 and
            not line.isdigit() and
            not line.startswith(('Abstract', 'Introduction', 'Page', 'arXiv:', 'doi:', 'http')) and
            ('University' in line or 'Institute' in line or
             ',' in line or 'Department' in line or
             '@' in line or  # Email addresses often indicate authors
             any(char.isupper() for char in line))):  # Names often have capitals

            # Clean up author line
            authors = line.strip()

            # Remove common prefixes
            prefixes_to_remove = ['Authors:', 'By:', 'Author:']
            for prefix in prefixes_to_remove:
                if authors.startswith(prefix):
                    authors = authors[len(prefix):].strip()

            # Truncate if too long
            if len(authors) > 150:
                authors = authors[:150].strip() + "..."

            return authors

    return "Unknown Authors"

print("📝 Title and Author extraction functions loaded successfully!")


📝 Title and Author extraction functions loaded successfully!


## 8. Search and Query Functions

Now let's create functions to search through our academic papers and extract detailed information about sources and metadata.


In [ ]:
# Updated list_indexed_papers function with title extraction
def list_indexed_papers_improved(documents: List) -> List[Dict[str, any]]:
    """
    List all papers that have been indexed with their metadata.
    Extracts actual paper titles and authors from document content.

    Args:
        documents (List): List of loaded documents

    Returns:
        List[Dict[str, any]]: List of paper information
    """
    papers = []
    processed_files = set()  # Track unique files to avoid duplicates

    for doc in documents:
        try:
            metadata = doc.metadata
            file_path = metadata.get("file_path", "")
            file_name = Path(file_path).stem if file_path else "Unknown"

            # Skip if we've already processed this file
            if file_path in processed_files:
                continue
            processed_files.add(file_path)

            # Extract title and authors from document text
            doc_text = doc.text if hasattr(doc, 'text') else ""
            extracted_title = extract_paper_title_from_text(doc_text)
            extracted_authors = extract_paper_authors_from_text(doc_text)

            paper_info = {
                "file_name": file_name,
                "file_path": file_path,
                "title": extracted_title,
                "authors": extracted_authors,
                "page_count": metadata.get("page_count", 0),
                "file_size": metadata.get("file_size", 0),
                "file_size_mb": round(metadata.get("file_size", 0) / (1024 * 1024), 2) if metadata.get("file_size") else 0,
                "total_pages": metadata.get("total_pages", "Unknown"),
                "page_label": metadata.get("page_label", ""),
            }

            papers.append(paper_info)

        except Exception as e:
            print(f"Error processing document: {e}")

    return papers

# Re-list papers with improved title extraction
print("🔄 Re-processing papers with improved title extraction...")
papers_list_improved = list_indexed_papers_improved(documents)

print(f"📚 Found {len(papers_list_improved)} unique papers in the index:")
print("=" * 80)

for i, paper in enumerate(papers_list_improved[:10], 1):  # Show first 10 papers
    print(f"{i}. 📄 {paper['title']}")
    print(f"   👥 Authors: {paper['authors']}")
    print(f"   📁 File: {paper['file_name']}")
    print(f"   💾 Size: {paper['file_size_mb']} MB")
    if paper.get('page_label'):
        print(f"   📖 Page: {paper['page_label']}")
    print("-" * 60)

print(f"\n✅ Successfully extracted titles for {len(papers_list_improved)} papers!")
if len(papers_list_improved) > 10:
    print(f"📝 Showing first 10 papers. Total: {len(papers_list_improved)} papers available.")


🔄 Re-processing papers with improved title extraction...
📚 Found 5 unique papers in the index:
1. 📄 AI Agents vs. Agentic AI: A Conceptual
   👥 Authors: Taxonomy, Applications and Challenges
   📁 File: AI_Agents_vs_Agentic_AI
   💾 Size: 3.05 MB
   📖 Page: 1
------------------------------------------------------------
2. 📄 THE LANDSCAPE OF EMERGING AI AGENT ARCHITECTURES
   👥 Authors: FOR REASONING , PLANNING , AND TOOL CALLING : A S URVEY
   📁 File: Emerging_Agent_Architectures
   💾 Size: 1.58 MB
   📖 Page: 1
------------------------------------------------------------
3. 📄 From LLM Reasoning to Autonomous AI Agents:
   👥 Authors: From LLM Reasoning to Autonomous AI Agents:
   📁 File: LLMReasoning_to_Autonomous_Agents
   💾 Size: 16.21 MB
   📖 Page: 1
------------------------------------------------------------
4. 📄 The Rise and Potential of Large Language Model
   👥 Authors: Based Agents: A Survey
   📁 File: Rise_and_Potential_LLM_Agents
   💾 Size: 6.52 MB
   📖 Page: 1
----------------

In [ ]:
# Enhanced search function with improved metadata extraction
def search_papers_improved(query_engine, query: str, include_metadata: bool = True) -> Dict[str, any]:
    """
    Search for relevant papers based on the query with improved title extraction.

    Args:
        query_engine: The configured query engine
        query (str): Search query
        include_metadata (bool): Whether to include detailed metadata

    Returns:
        Dict[str, any]: Search results with response and sources
    """
    if not query_engine:
        return {
            "success": False,
            "error": "Query engine not initialized.",
            "response": "",
            "sources": [],
        }

    try:
        print(f"🔍 Searching for: '{query}'")
        start_time = time.time()

        # Query the RAG system
        response = query_engine.query(query)

        end_time = time.time()

        # Extract source information from retrieved nodes with title extraction
        sources = []
        if hasattr(response, "source_nodes"):
            for node in response.source_nodes:
                # Extract title from the node text
                node_text = node.text if hasattr(node, 'text') else ""
                extracted_title = extract_paper_title_from_text(node_text, max_length=100)
                extracted_authors = extract_paper_authors_from_text(node_text)

                source_info = {
                    "text": (
                        node.text[:500] + "..."
                        if len(node.text) > 500
                        else node.text
                    ),
                    "score": getattr(node, "score", 0.0),
                    "extracted_title": extracted_title,
                    "extracted_authors": extracted_authors,
                }

                # Add metadata if available and requested
                if include_metadata and hasattr(node, "metadata"):
                    metadata = node.metadata
                    source_info.update({
                        "file_name": metadata.get("file_name", "Unknown"),
                        "file_path": metadata.get("file_path", ""),
                        "page_label": metadata.get("page_label", ""),
                        "file_size_mb": round(metadata.get("file_size", 0) / (1024 * 1024), 2) if metadata.get("file_size") else 0,
                    })

                sources.append(source_info)

        result = {
            "success": True,
            "response": str(response),
            "sources": sources,
            "query": query,
            "search_time": end_time - start_time,
            "num_sources": len(sources),
        }

        print(f"✓ Search completed in {end_time - start_time:.2f} seconds")
        print(f"📚 Found {len(sources)} relevant sources")

        return result

    except Exception as e:
        print(f"❌ Error during search: {e}")
        return {"success": False, "error": str(e), "response": "", "sources": []}

print("🔍 Enhanced search function with title extraction loaded!")


🔍 Enhanced search function with title extraction loaded!


In [ ]:
# Test the improved search with title extraction
def ask_question_improved(query_engine, question: str, show_sources: bool = True):
    """
    Ask a custom question to the RAG system with improved title extraction.

    Args:
        query_engine: The configured query engine
        question (str): Your question about the papers
        show_sources (bool): Whether to display source information
    """
    print(f"❓ Question: {question}")
    print("=" * 80)

    result = search_papers_improved(query_engine, question, include_metadata=True)

    if result["success"]:
        print(f"💡 Answer:")
        print(result["response"])
        print(f"\n📊 Search completed in {result['search_time']:.2f} seconds")
        print(f"📚 Found {result['num_sources']} relevant sources")

        if show_sources and result["sources"]:
            print(f"\n📖 Source Details:")
            print("-" * 60)
            for i, source in enumerate(result["sources"], 1):
                print(f"\n{i}. 📄 Paper: {source.get('extracted_title', 'Unknown Title')}")
                print(f"   👥 Authors: {source.get('extracted_authors', 'Unknown Authors')}")
                print(f"   📁 File: {source.get('file_name', 'Unknown')}")
                if source.get('page_label'):
                    print(f"   📖 Page: {source['page_label']}")
                print(f"   🎯 Relevance Score: {source.get('score', 0):.3f}")
                print(f"   📝 Text Preview: {source['text'][:200]}...")

    else:
        print(f"❌ Error: {result['error']}")

# Test with the improved version
test_question = "What are the main architectural patterns for agent systems?"

if query_engine:
    print("🧪 Testing improved search with title extraction:")
    print("=" * 80)
    ask_question_improved(query_engine, test_question, show_sources=True)
else:
    print("❌ Query engine not available")


🧪 Testing improved search with title extraction:
❓ Question: What are the main architectural patterns for agent systems?
🔍 Searching for: 'What are the main architectural patterns for agent systems?'
✓ Search completed in 3.08 seconds
📚 Found 5 relevant sources
💡 Answer:
The main architectural patterns for agent systems include single-agent and multi-agent architectures. Single-agent patterns focus on individual agents that may receive feedback from humans but not from other agents. Multi-agent architectures involve multiple agents that can either use the same or different language models and tools. These architectures are categorized into vertical and horizontal structures. Vertical architectures have a lead agent with other agents reporting to it, while horizontal architectures treat all agents as equals, allowing them to communicate in a shared discussion and volunteer for tasks.

📊 Search completed in 3.08 seconds
📚 Found 5 relevant sources

📖 Source Details:
----------------------

In [ ]:
def search_papers(query_engine, query: str, include_metadata: bool = True) -> Dict[str, any]:
    """
    Search for relevant papers based on the query.

    Args:
        query_engine: The configured query engine
        query (str): Search query
        include_metadata (bool): Whether to include detailed metadata

    Returns:
        Dict[str, any]: Search results with response and sources
    """
    if not query_engine:
        return {
            "success": False,
            "error": "Query engine not initialized.",
            "response": "",
            "sources": [],
        }

    try:
        print(f"🔍 Searching for: '{query}'")
        start_time = time.time()

        # Query the RAG system
        response = query_engine.query(query)

        end_time = time.time()

        # Extract source information from retrieved nodes
        sources = []
        if hasattr(response, "source_nodes"):
            for node in response.source_nodes:
                source_info = {
                    "text": (
                        node.text[:500] + "..."
                        if len(node.text) > 500
                        else node.text
                    ),
                    "score": getattr(node, "score", 0.0),
                }

                # Add metadata if available and requested
                if include_metadata and hasattr(node, "metadata"):
                    metadata = node.metadata
                    source_info.update({
                        "file_name": metadata.get("file_name", "Unknown"),
                        "title": metadata.get("title", "Unknown Title"),
                        "authors": metadata.get("authors", "Unknown Authors"),
                        "page_count": metadata.get("page_count", 0),
                        "has_abstract": metadata.get("has_abstract", False),
                    })

                sources.append(source_info)

        result = {
            "success": True,
            "response": str(response),
            "sources": sources,
            "query": query,
            "search_time": end_time - start_time,
            "num_sources": len(sources),
        }

        print(f"✓ Search completed in {end_time - start_time:.2f} seconds")
        print(f"📚 Found {len(sources)} relevant sources")

        return result

    except Exception as e:
        print(f"❌ Error during search: {e}")
        return {"success": False, "error": str(e), "response": "", "sources": []}

# Test the search function with a sample query
test_query = "What are the main types of AI agents discussed in these papers?"
result = search_papers(query_engine, test_query)

if result["success"]:
    print(f"\n📝 Response Preview: {result['response']}")
    print(f"📊 Number of sources: {result['num_sources']}")
    print(f"⏱️  Search time: {result['search_time']:.2f} seconds")
else:
    print(f"❌ Search failed: {result['error']}")


🔍 Searching for: 'What are the main types of AI agents discussed in these papers?'
✓ Search completed in 4.84 seconds
📚 Found 5 relevant sources

📝 Response Preview: The papers discuss various types of AI agents, including retrieval-augmented generative agents for scientific research, models as agents in multi-agent reinforcement learning, multi-agent embodied AI, and single and multi-agent deep reinforcement learning for AI-enabled wireless networks. Additionally, they explore agentic AI in intelligent communications and networking.
📊 Number of sources: 5
⏱️  Search time: 4.84 seconds


## 9. Paper Information and Metadata

Let's create functions to list and get detailed information about the papers in our index.


In [ ]:
def list_indexed_papers(documents: List) -> List[Dict[str, any]]:
    """
    List all papers that have been indexed with their metadata.

    Args:
        documents (List): List of loaded documents

    Returns:
        List[Dict[str, any]]: List of paper information
    """
    papers = []

    for doc in documents:
        try:
            metadata = doc.metadata
            file_path = metadata.get("file_path", "")
            file_name = Path(file_path).stem if file_path else "Unknown"

            paper_info = {
                "file_name": file_name,
                "file_path": file_path,
                "title": metadata.get("title", file_name),
                "authors": metadata.get("authors", "Unknown"),
                "page_count": metadata.get("page_count", 0),
                "file_size": metadata.get("file_size", 0),
                "file_size_mb": round(metadata.get("file_size", 0) / (1024 * 1024), 2) if metadata.get("file_size") else 0,
                "total_pages": metadata.get("total_pages", "Unknown"),
            }

            papers.append(paper_info)

        except Exception as e:
            print(f"Error processing document: {e}")

    return papers

# List all indexed papers
papers_list = list_indexed_papers(documents)

print(f"📚 Found {len(papers_list)} papers in the index:")
print("=" * 60)

for i, paper in enumerate(papers_list, 1):
    print(f"{i}. {paper['file_name']}")
    print(f"   File Path: {paper['file_path']}")
    print(f"   Total Pages: {paper.get('total_pages', 'Unknown')}")
    print(f"   Size: {paper['file_size_mb']} MB")
    print("-" * 40)


📚 Found 229 papers in the index:
1. AI_Agents_vs_Agentic_AI
   File Path: /content/drive/MyDrive/Colab Notebooks/papers-agents/AI_Agents_vs_Agentic_AI.pdf
   Total Pages: Unknown
   Size: 3.05 MB
----------------------------------------
2. AI_Agents_vs_Agentic_AI
   File Path: /content/drive/MyDrive/Colab Notebooks/papers-agents/AI_Agents_vs_Agentic_AI.pdf
   Total Pages: Unknown
   Size: 3.05 MB
----------------------------------------
3. AI_Agents_vs_Agentic_AI
   File Path: /content/drive/MyDrive/Colab Notebooks/papers-agents/AI_Agents_vs_Agentic_AI.pdf
   Total Pages: Unknown
   Size: 3.05 MB
----------------------------------------
4. AI_Agents_vs_Agentic_AI
   File Path: /content/drive/MyDrive/Colab Notebooks/papers-agents/AI_Agents_vs_Agentic_AI.pdf
   Total Pages: Unknown
   Size: 3.05 MB
----------------------------------------
5. AI_Agents_vs_Agentic_AI
   File Path: /content/drive/MyDrive/Colab Notebooks/papers-agents/AI_Agents_vs_Agentic_AI.pdf
   Total Pages: Unknown
   Si

## 11. Advanced Query Examples

Now let's test our RAG system with various types of research queries to demonstrate its capabilities.


In [ ]:
def run_example_queries(query_engine):
    """
    Run a series of example queries to demonstrate RAG capabilities.

    Args:
        query_engine: The configured query engine
    """
    example_queries = [
        {
            "category": "Agent Types",
            "query": "What are the main types of AI agents discussed in these papers?",
        },
        {
            "category": "Technical Comparison",
            "query": "How do LLM-based agents differ from traditional AI agents?",
        },
        {
            "category": "Challenges",
            "query": "What are the current challenges in developing autonomous agents?",
        },
        {
            "category": "Evaluation",
            "query": "What evaluation methods are used for AI agents?",
        },
        {
            "category": "Architecture",
            "query": "Describe the common architectural patterns for agent systems.",
        },
        {
            "category": "Applications",
            "query": "What are the practical applications of AI agents mentioned in the literature?",
        },
    ]

    print("🧪 Running Example Queries")
    print("=" * 60)

    for i, example in enumerate(example_queries, 1):
        print(f"\n{i}. {example['category']}")
        print(f"Q: {example['query']}")
        print("-" * 50)

        result = search_papers(query_engine, example["query"])

        if result["success"]:
            # Display truncated response
            response = result["response"]
            if len(response) > 400:
                response = response[:400] + "..."

            print(f"A: {response}")
            print(f"📚 Sources: {result['num_sources']} | ⏱️  Time: {result['search_time']:.2f}s")
        else:
            print(f"❌ Error: {result['error']}")

        print()

# Run the example queries
if query_engine:
    run_example_queries(query_engine)
else:
    print("❌ Query engine not available for examples")


🧪 Running Example Queries

1. Agent Types
Q: What are the main types of AI agents discussed in these papers?
--------------------------------------------------
🔍 Searching for: 'What are the main types of AI agents discussed in these papers?'
✓ Search completed in 5.63 seconds
📚 Found 5 relevant sources
A: The papers discuss various types of AI agents, including retrieval-augmented generative agents for scientific research, models as agents in multi-agent reinforcement learning, multi-agent embodied AI, and single and multi-agent deep reinforcement learning for AI-enabled wireless networks. Additionally, they cover agentic AI in intelligent communications and networking.
📚 Sources: 5 | ⏱️  Time: 5.63s


2. Technical Comparison
Q: How do LLM-based agents differ from traditional AI agents?
--------------------------------------------------
🔍 Searching for: 'How do LLM-based agents differ from traditional AI agents?'
✓ Search completed in 2.49 seconds
📚 Found 5 relevant sources
A: LLM-bas

## 12. Interactive Query Interface

Let's create an interactive function that allows you to ask custom questions about the papers.


In [ ]:
def ask_question(query_engine, question: str, show_sources: bool = True):
    """
    Ask a custom question to the RAG system and display results.

    Args:
        query_engine: The configured query engine
        question (str): Your question about the papers
        show_sources (bool): Whether to display source information
    """
    print(f"❓ Question: {question}")
    print("=" * 60)

    result = search_papers(query_engine, question, include_metadata=True)

    if result["success"]:
        print(f"💡 Answer:")
        print(result["response"])
        print(f"\n📊 Search completed in {result['search_time']:.2f} seconds")
        print(f"📚 Found {result['num_sources']} relevant sources")

        if show_sources and result["sources"]:
            print(f"\n📖 Source Details:")
            print("-" * 40)
            for i, source in enumerate(result["sources"], 1):
                print(f"\n{i}. Score: {source.get('score', 0):.3f}")
                print(f"   Text: {source['text'][:200]}...")

    else:
        print(f"❌ Error: {result['error']}")

# Example usage - you can modify this question
custom_question = "What are the key ethical considerations for AI agents?"

if query_engine:
    ask_question(query_engine, custom_question, show_sources=True)
else:
    print("❌ Query engine not available")


❓ Question: What are the key ethical considerations for AI agents?
🔍 Searching for: 'What are the key ethical considerations for AI agents?'
✓ Search completed in 5.39 seconds
📚 Found 5 relevant sources
💡 Answer:
Key ethical considerations for AI agents include accountability, oversight, and value alignment. In multi-agent environments, defining accountability is crucial to prevent gaps when agents collaborate, as this can complicate responsibility assignment. Ensuring fairness is essential, particularly when agents are trained on biased data, as this can perpetuate systemic inequities. Value alignment is vital to maintain consistency between agents' actions and human intentions and ethical standards, especially in dynamic settings. Ethical governance frameworks and oversight mechanisms are necessary to address these challenges effectively.

📊 Search completed in 5.39 seconds
📚 Found 5 relevant sources

📖 Source Details:
----------------------------------------

1. Score: 0.647
   Text

## 13. System Performance and Statistics

Let's create functions to analyze and display performance statistics of our RAG system.


In [ ]:
def display_system_stats(papers_list, vector_store, index):
    """
    Display comprehensive statistics about the RAG system.

    Args:
        papers_list: List of indexed papers
        vector_store: The vector store instance
        index: The vector index
    """
    print("📊 RAG System Statistics")
    print("=" * 50)

    # Paper statistics
    total_papers = len(papers_list)
    total_pages = sum(paper.get('page_count', 0) for paper in papers_list)
    total_size_mb = sum(paper.get('file_size_mb', 0) for paper in papers_list)

    print(f"📚 Document Statistics:")
    print(f"   Total Papers: {total_papers}")
    print(f"   Total Pages: {total_pages}")
    print(f"   Total Size: {total_size_mb:.2f} MB")
    print(f"   Average Pages per Paper: {total_pages/total_papers:.1f}" if total_papers > 0 else "   Average Pages: N/A")

    # Index statistics
    if index:
        print(f"\n🗂️  Index Statistics:")
        print(f"   Index Type: Vector Store Index")
        print(f"   Embedding Model: {get_config('api.openai.embedding_model', 'text-embedding-3-small')}")
        print(f"   LLM Model: {get_config('api.openai.model', 'gpt-4o-mini')}")

    # Storage paths
    print(f"\n💾 Storage Locations:")
    print(f"   Papers Folder: papers/agents")
    print(f"   Vector Database: storage/papers_vectordb")
    print(f"   Index Storage: storage/papers_index")

    # Recent papers by modification time
    if papers_list:
        print(f"\n📋 Paper Titles:")
        for i, paper in enumerate(papers_list, 1):
            title = paper['title']
            if len(title) > 50:
                title = title[:47] + "..."
            print(f"   {i}. {title}")

# Display system statistics
display_system_stats(papers_list, vector_store, index)
print("\n✅ RAG System Analysis Complete!")


📊 RAG System Statistics
📚 Document Statistics:
   Total Papers: 229
   Total Pages: 0
   Total Size: 1962.34 MB
   Average Pages per Paper: 0.0

🗂️  Index Statistics:
   Index Type: Vector Store Index
   Embedding Model: text-embedding-3-small
   LLM Model: gpt-4o-mini

💾 Storage Locations:
   Papers Folder: papers/agents
   Vector Database: storage/papers_vectordb
   Index Storage: storage/papers_index

📋 Paper Titles:
   1. AI_Agents_vs_Agentic_AI
   2. AI_Agents_vs_Agentic_AI
   3. AI_Agents_vs_Agentic_AI
   4. AI_Agents_vs_Agentic_AI
   5. AI_Agents_vs_Agentic_AI
   6. AI_Agents_vs_Agentic_AI
   7. AI_Agents_vs_Agentic_AI
   8. AI_Agents_vs_Agentic_AI
   9. AI_Agents_vs_Agentic_AI
   10. AI_Agents_vs_Agentic_AI
   11. AI_Agents_vs_Agentic_AI
   12. AI_Agents_vs_Agentic_AI
   13. AI_Agents_vs_Agentic_AI
   14. AI_Agents_vs_Agentic_AI
   15. AI_Agents_vs_Agentic_AI
   16. AI_Agents_vs_Agentic_AI
   17. AI_Agents_vs_Agentic_AI
   18. AI_Agents_vs_Agentic_AI
   19. AI_Agents_vs_Agentic

## Conclusion

🎉 **Congratulations!** You have successfully built a complete RAG (Retrieval-Augmented Generation) system for academic papers using LlamaIndex.

### What we accomplished:

1. **Environment Setup**: Configured API keys and dependencies
2. **Configuration Management**: Loaded system settings from YAML files
3. **LlamaIndex Configuration**: Set up embeddings, LLM, and text processing
4. **Vector Store**: Created a LanceDB vector database for storing embeddings
5. **Document Processing**: Loaded and processed PDF academic papers
6. **Vector Indexing**: Created searchable vector embeddings of documents
7. **Query Engine**: Set up retrieval and response generation
8. **Search Functions**: Implemented semantic search with metadata
9. **Paper Analysis**: Created functions for listing and summarizing papers
10. **Interactive Queries**: Built an interface for asking custom questions
11. **Performance Analytics**: Added system statistics and monitoring

### Key Features:

- **Semantic Search**: Find relevant content using natural language queries
- **Source Attribution**: Get detailed citations and references for answers
- **Metadata Integration**: Access paper titles, authors, and other metadata
- **Performance Monitoring**: Track search times and system statistics
- **Flexible Configuration**: Easy to modify models, chunk sizes, and parameters

### Next Steps:

1. **Experiment** with different queries to explore your document collection
2. **Modify** the `custom_question` variable to ask your own questions
3. **Adjust** parameters like `chunk_size`, `similarity_top_k` for different results
4. **Add** more papers to the `papers/agents` folder and rebuild the index
5. **Enhance** the system with additional features like filtering or ranking

### Usage Tips:

- Use specific, focused questions for better results
- Try different phrasings of the same question
- Check the source information to understand where answers come from
- Experiment with the `similarity_top_k` parameter to get more or fewer sources

Happy researching! 🔬📚
